# Create a table with SRA IDs, readstats QC data, and submitter metadata

**Below are the steps taken in this notebook:**
1. Installs, import Statements & Global Variable Definitions
2. Read in tables as dataframes
3. Merge dataframes
4. Write merged dataframe to data table and file

**Important Note**  
This may not be the best way to aggregate the data, because SRA submissions are done after QC, which means the workspace needs to be kept and run with newly uploaded info. It might be easier to just output the Readstats table and merge outside Terra/Anvil


## Installs

In [1]:
%%capture
%pip install gcsfs
## capture CANNOT have comments above it
## For reading CSVs stored in Google Cloud (without downloading them first)
## May need to restart kernel after install 

In [2]:
%%capture
%pip install --upgrade --no-cache-dir --force-reinstall terra-pandas
%pip install --upgrade --no-cache-dir  --force-reinstall git+https://github.com/DataBiosphere/terra-notebook-utils
## For reading/writing data tables into pandas data frames
## May need to restart kernel after install 

## Import Statements

In [3]:
from firecloud import fiss
import pandas as pd      
import os                 
import subprocess       
import re                 
import io
import gcsfs

from typing import Any, Callable, List, Optional
from terra_notebook_utils import table, WORKSPACE_NAME, WORKSPACE_GOOGLE_PROJECT
from terra_pandas import dataframe_to_table, table_to_dataframe

## Global Variable Declarations

In [4]:
# AnVIL_HPRC WorkspaceBucket
anvil_hprc_bucket       = "gs://fc-285575a2-c2d5-4f17-a6a1-d1811d4afab4/"

# table filenames (expected in the workspace bucket, see below)
submitter_metadata = 'DEEPCONSENSUS_v1pt2_submitter_metadata.txt'
sra_metadata = 'metadata-13754908-processed-ok.tsv'

# submission id
submission_id = 'DEEPCONSENSUS_v1pt2'

# Get the Google billing project name and workspace name for current workspace
PROJECT = os.environ['WORKSPACE_NAMESPACE']
WORKSPACE =os.path.basename(os.path.dirname(os.getcwd()))
bucket = os.environ['WORKSPACE_BUCKET'] + "/"

# Verify that we've captured the environment variables
print("Billing project: " + PROJECT)
print("Workspace: " + WORKSPACE)
print("Workspace storage bucket: " + bucket)

Billing project: human-pangenome-ucsc
Workspace: HPRC_WRANGLING_DEEPCONSENSUS_v1pt2
Workspace storage bucket: gs://fc-285575a2-c2d5-4f17-a6a1-d1811d4afab4/


# Read In tables as dataframes

## Readstats table
Find the individual readstats output files in the table and concatenate their contents.
Add the hifi filename and sample name.

In [5]:
# readstats
readstats_df = table_to_dataframe("readstats", workspace=WORKSPACE, workspace_namespace=PROJECT)
qc_list = []

for index, row in readstats_df.iterrows():

        sample_readstats_fp = row['ReadStatsReport']
        sample_readstats_fn = os.path.basename(sample_readstats_fp)
        df = pd.read_csv(sample_readstats_fp, header=None, sep='\t')
        df = df[df[0]=='sample.fastq']
        # add sample and file name
        #df = df.append({0:'undef', 1:'sample', 2:row['sample']}, ignore_index=True)
        df = df.append({0:'undef', 1:'filename', 2:os.path.basename(row['hifi'])}, ignore_index=True)
        # remove all but the variables (total_bp, quartile_25 etc)
        df2 = df.drop(df.columns[[0, 1]], axis=1)
        # make rownames ('total_bp', 'quartile_25' etc)
        df2.index = df.iloc[:,1]
        # remove the now meaningless 'sample.fastq' filename
        df2 = df2.drop(index=['file'])
        qc_list.append(df2)

# merge
readstats_df = pd.concat(qc_list, axis=1).transpose()
readstats_df.head()

1,total_reads,total_bp,total_Gbp,min,max,mean,quartile_25,quartile_50,quartile_75,N25,N50,N75,filename
2,2169288,40034895477,40.03,43,49368,18455,16373,18217,20127,16798,18643,20614,HG00423.m64043_210205_190424.dc.q20.fastq.gz
2,2102877,38839139125,38.84,25,50437,18469,16369,18215,20151,16793,18648,20653,HG00423.m64043_210207_011920.dc.q20.fastq.gz
2,1917582,39047683785,39.05,47,50778,20362,17437,19156,22527,17779,19886,23774,HG00609.m64136_210622_163607.dc.q20.fastq.gz
2,1835281,34913876902,34.91,44,49808,19023,16246,18055,21265,16586,18981,22124,HG03654.m64043_210524_174752.dc.q20.fastq.gz
2,1435262,27331360113,27.33,722,50298,19042,16278,18102,21275,16617,19019,22123,HG03654.m64043_210528_171552.dc.q20.fastq.gz


## Submitter metadata table

In [8]:
meta_fp = os.path.join(bucket, submitter_metadata)
metadata_df = pd.read_csv(meta_fp, sep='\t')
metadata_df = metadata_df[['filename', 'sample_ID', 'instrument_model', 'shear_method', 'size_selection', 'DeepConsensus_version', 
                           'polymerase_version', 'seq_plate_chemistry_version', 'generator_facility', 'generator_contact']]
metadata_df.head()

,filename,sample_ID,instrument_model,shear_method,size_selection,DeepConsensus_version,polymerase_version,seq_plate_chemistry_version,generator_facility,generator_contact
0,HG00423.m64043_210205_190424.dc.q20.fastq.gz,HG00423,PacBio Sequel II,Megaruptor 1,SageELF,1.2,P2,C2,Washington University,tgraves@wustl.edu
1,HG00423.m64043_210207_011920.dc.q20.fastq.gz,HG00423,PacBio Sequel II,Megaruptor 1,SageELF,1.2,P2,C2,Washington University,tgraves@wustl.edu
2,HG00423.m64043_210209_184051.dc.q20.fastq.gz,HG00423,PacBio Sequel II,Megaruptor 1,SageELF,1.2,P2,C2,Washington University,tgraves@wustl.edu
3,HG00423.m64043_210211_005516.dc.q20.fastq.gz,HG00423,PacBio Sequel II,Megaruptor 1,SageELF,1.2,P2,C2,Washington University,tgraves@wustl.edu
4,HG00544.m64043_210702_183246.dc.q20.fastq.gz,HG00544,PacBio Sequel II,Megaruptor 1,SageELF,1.2,P2,C2,Washington University,tgraves@wustl.edu


## SRA table
Must download after finishing submission.
The downloaded table needs to be split in one row per file (instead of filename, filename2 <...> filenameN)

In [23]:
## Read in SRA file and split by filename

sra_file = os.path.join(bucket + sra_metadata)
sample_df = pd.read_csv(sra_file, sep='\t')

fnames = [x for x in sample_df.columns if x.startswith('filename')]
sra_df = pd.melt(sample_df, id_vars =['library_ID','accession'], value_vars = fnames, value_name='file')
sra_df= sra_df.dropna()
sra_df = sra_df[['file', 'accession', 'library_ID']]
sra_df.columns = ['filename', 'accession', 'library_ID']
sra_df.head()


,filename,accession,library_ID
0,HG00423.m64043_210205_190424.dc.q20.fastq.gz,SRR25618944,HG00423_lib1_dc
1,HG00544.m64136_210520_180433.dc.q20.fastq.gz,SRR25618943,HG00544_lib1_dc
2,HG00544.m64043_210702_183246.dc.q20.fastq.gz,SRR25618932,HG00544_lib2_dc
3,HG00609.m64136_210622_163607.dc.q20.fastq.gz,SRR25618921,HG00609_lib1_dc
4,HG00642.m54329U_210524_174516.dc.q20.fastq.gz,SRR25618910,HG00642.HFSS_dc


# Merge all dataframes

In [14]:
print(sra_df.shape, readstats_df.shape, metadata_df.shape)
# two samples of HG02572 were not uploaded to SRA with the rest of this batch

(132, 3) (132, 13) (132, 10)


In [15]:
set(readstats_df['filename']) - set(sra_df['filename'])
#m54329U_201103_231616.ccs.bam (HG002 sample) is not present in the submitter metadata or on SRA, but it is in the submissions 

set()

In [16]:
merged_df = pd.merge(
    sra_df,
    readstats_df,
    on='filename')
merged_df = pd.merge(
    merged_df,
    metadata_df,
    on='filename')
merged_df.head()

,filename,accession,library_ID,total_reads,total_bp,total_Gbp,min,max,mean,quartile_25,...,N75,sample_ID,instrument_model,shear_method,size_selection,DeepConsensus_version,polymerase_version,seq_plate_chemistry_version,generator_facility,generator_contact
0,HG00423.m64043_210205_190424.dc.q20.fastq.gz,SRR25618944,HG00423_lib1_dc,2169288,40034895477,40.03,43,49368,18455,16373,...,20614,HG00423,PacBio Sequel II,Megaruptor 1,SageELF,1.2,P2,C2,Washington University,tgraves@wustl.edu
1,HG00544.m64136_210520_180433.dc.q20.fastq.gz,SRR25618943,HG00544_lib1_dc,1622000,31807341724,31.81,48,50293,19609,16831,...,22870,HG00544,PacBio Sequel II,Megaruptor 1,SageELF,1.2,P2,C2,Washington University,tgraves@wustl.edu
2,HG00544.m64043_210702_183246.dc.q20.fastq.gz,SRR25618932,HG00544_lib2_dc,2258284,42595473911,42.60,473,48786,18861,17346,...,20289,HG00544,PacBio Sequel II,Megaruptor 1,SageELF,1.2,P2,C2,Washington University,tgraves@wustl.edu
3,HG00609.m64136_210622_163607.dc.q20.fastq.gz,SRR25618921,HG00609_lib1_dc,1917582,39047683785,39.05,47,50778,20362,17437,...,23774,HG00609,PacBio Sequel II,Megaruptor 1,SageELF,1.2,P2,C2,Washington University,tgraves@wustl.edu
4,HG00642.m54329U_210524_174516.dc.q20.fastq.gz,SRR25618910,HG00642.HFSS_dc,2098346,41901198683,41.90,3,50177,19968,16616,...,23857,HG00642,PacBio Sequel II,Megaruptor 3,BluePippin,1.2,P2,C2,University of Washington,kmiyamot@uw.edu


# Write output files

In [24]:
## Create table
#dataframe_to_table("post_sra_metadata", merged_df, WORKSPACE, PROJECT)
outf = os.path.join(bucket, submission_id + '_post_sra_metadata.tsv')
merged_df.to_csv(outf, sep="\t", index=False)

In [25]:
merged_df.shape

(132, 24)